In [1]:
from dotenv import load_dotenv
from neo4j import GraphDatabase
import os
## Load environment variables from .env file
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
AUTH = (user, password)

In [3]:
from neo4j import GraphDatabase


# --- prerequisites ------------------------------------------------------
driver   = GraphDatabase.driver(uri, auth=(user, password))


## Retriever = Approach - 1

In [35]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
embedder = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-large")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=api_key)

In [5]:
from openai import OpenAI
import json

client = OpenAI(api_key=api_key)  # ideally os.getenv()

# -------------------- Entity Extraction -------------------- #
def extract_entities(query: str) -> list[str]:
    tools = [
        {
            "type": "function",
            "function": {
                "name": "extract_entities",
                "description": "Extracts all proper nouns or named entities from a query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "entities": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of named entities"
                        }
                    },
                    "required": ["entities"]
                }
            }
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Extract named entities from the user question."},
            {"role": "user", "content": query}
        ],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "extract_entities"}}
    )

    args = response.choices[0].message.tool_calls[0].function.arguments
    return json.loads(args)["entities"]


In [6]:
# -------------------- Query Intent Classification -------------------- #
def classify_intent(query: str) -> str:
    tools = [
        {
            "type": "function",
            "function": {
                "name": "classify_intent",
                "description": "Classifies the query intent as fact or context.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "intent": {
                            "type": "string",
                            "enum": ["fact", "context"],
                            "description": "Is it a fact query (asks for a list or factual triple) or a context query (asks for explanation)?"
                        }
                    },
                    "required": ["intent"]
                }
            }
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Classify the user query as 'fact' or 'context'."},
            {"role": "user", "content": query}
        ],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "classify_intent"}}
    )

    args = response.choices[0].message.tool_calls[0].function.arguments
    return json.loads(args)["intent"]


In [7]:
query = "List all iPhones released by Apple in 2022"

entities = extract_entities(query)
intent = classify_intent(query)

print("Entities:", entities)
print("Intent:", intent)

Entities: ['iPhones', 'Apple', '2022']
Intent: fact


In [8]:
from typing import List, Dict, Tuple, Callable, Optional, Sequence

class TripleRetriever:
    def __init__(self, driver: GraphDatabase.driver):
        self.driver = driver

    def get_triples(self, query_entities: List[str], limit: int = 20) -> List[Dict]:
        if not query_entities:
            return []

        cypher = """
        MATCH (s)-[r]->(o)
        WHERE toLower(s.name) IN $entities OR toLower(o.name) IN $entities
        RETURN s.name AS subject, type(r) AS relation, o.name AS object
        LIMIT $limit
        """
        with self.driver.session() as session:
            rows = session.run(
                cypher,
                entities=[e.lower() for e in query_entities],
                limit=limit,
            ).data()
        return rows  # List of {"subject": ..., "relation": ..., "object": ...}


In [9]:
# hybrid_retriever.py
from __future__ import annotations
from typing import List, Dict, Tuple, Callable, Optional, Sequence
from collections import defaultdict

import numpy as np
from neo4j import GraphDatabase
from langchain.embeddings.base import Embeddings


class HybridRetriever:
    """
    A *single-entry-point* retriever that combines

    1. **Dense semantic search** over `(:Chunk)` vectors stored in Neo4j
    2. **Structural expansion** (PARENT_OF, NEXT_CHUNK) to keep narrative context
    3. **Knowledge-graph probing** via light-weight NER →  `(:Entity)-[:MENTIONED_IN]->(:Chunk)`
    4. **Reciprocal-Rank Fusion** to merge the three candidate sets

    The end product is an *ordered* list of chunk dictionaries ready to be
    concatenated into your RAG prompt template.

    Parameters
    ----------
    driver :
        An active `neo4j.GraphDatabase.driver`.
    embedder :
        An object that implements `embed_query(text) -> List[float]`
        (e.g. `langchain_openai.OpenAIEmbeddings`).
    index_name :
        Name of the Neo4j vector index created by `create_vector_index`.
    entity_tagger :
        Callable that returns a list of surface strings found in a query.
        Default is a mini-regex fallback that picks capitalised noun-phrases;
        in production you will probably inject spaCy, HuggingFace, or an
        OpenAI function-calling wrapper.
    top_k_dense :
        How many dense seeds to return.
    expand_parents / expand_neighbors :
        How many *per-seed* hierarchical and adjacent chunks to pull in
        (0 disables the respective expansion).
    kg_chunk_limit :
        Per-entity cap when pulling chunks through the KG.
    max_chunks :
        Final cap after fusion/rerank – keeps your context window bounded.
    """

    def __init__(
        self,
        driver: GraphDatabase.driver,
        embedder: Embeddings,
        *,
        index_name: str,
        entity_tagger: Optional[Callable[[str], Sequence[str]]] = None,
        top_k_dense: int = 5,
        expand_parents: int = 1,
        expand_neighbors: int = 1,
        kg_chunk_limit: int = 3,
        max_chunks: int = 12,
    ) -> None:
        self.driver = driver
        self.embedder = embedder
        self.index_name = index_name
        self.top_k_dense = top_k_dense
        self.expand_parents = expand_parents
        self.expand_neighbors = expand_neighbors
        self.kg_chunk_limit = kg_chunk_limit
        self.max_chunks = max_chunks
        self.entity_tagger = entity_tagger or self._naive_capitalised_np

    # --------------------------------------------------------------------- #
    # 1) Dense search (vector index)                                         #
    # --------------------------------------------------------------------- #
    def _dense_seeds(self, query: str) -> Tuple[List[str], List[float]]:
        embedding = self.embedder.embed_query(query)
        cypher = (
            "CALL db.index.vector.queryNodes($index_name, $k, $embed) "
            "YIELD node, score "
            "RETURN node.id AS id, score "
            "ORDER BY score DESC"
        )
        with self.driver.session() as s:
            res = s.run(
                cypher,
                index_name=self.index_name,
                k=self.top_k_dense,
                embed=embedding,
            )
            rows = res.data()
        ids = [r["id"] for r in rows]
        scores = [r["score"] for r in rows]
        return ids, scores

    # --------------------------------------------------------------------- #
    # 2) Context expansion via PARENT_OF / NEXT_CHUNK                       #
    # --------------------------------------------------------------------- #
    def _expand_parents(self, chunk_ids: List[str], hops: int) -> set[str]:
        if hops == 0 or not chunk_ids:
            return set()
        cypher = f"""
            MATCH (c:Chunk)<-[:PARENT_OF*1..{hops}]-(p)
            WHERE c.id IN $ids
            RETURN DISTINCT p.id AS id
        """
        with self.driver.session() as s:
            rows = s.run(cypher, ids=chunk_ids).data()
        return {r["id"] for r in rows}


    def _expand_neighbors(self, chunk_ids: List[str], hops: int) -> set[str]:
        if hops == 0 or not chunk_ids:
            return set()
        cypher = f"""
            MATCH (c1:Chunk)-[:NEXT_CHUNK*1..{hops}]->(c2:Chunk)
            WHERE c1.id IN $ids
            RETURN DISTINCT c2.id AS id
        """
        with self.driver.session() as s:
            rows = s.run(cypher, ids=chunk_ids).data()
        return {r["id"] for r in rows}


    # --------------------------------------------------------------------- #
    # 3) Knowledge-graph probe                                              #
    # --------------------------------------------------------------------- #
    def kg_probe_from_entities(self, entities: List[str]) -> List[str]:
        if not entities:
            return []
        cypher = (
            "MATCH (e)-[:MENTIONED_IN]->(c:Chunk) "
            "WHERE toLower(e.name) IN $ents "
            "RETURN DISTINCT c.id AS id "
            "LIMIT $lim"
        )
        with self.driver.session() as s:
            rows = s.run(
                cypher,
                ents=[e.lower() for e in entities],
                lim=self.kg_chunk_limit * len(entities),
            ).data()
        return [r["id"] for r in rows]


    # --------------------------------------------------------------------- #
    # 4) Reciprocal-Rank-Fusion                                             #
    # --------------------------------------------------------------------- #
    def _rrf_fuse(self, runs: List[Tuple[str, List[str]]], k: int = 60) -> List[Tuple[str, float]]:
        """
        Each `runs[i]` is (label, [id0, id1, ...]).

        RRF score = 1 / (k + rank).  Higher is better.
        """
        scores = defaultdict(float)
        for _, ids in runs:
            for rank, cid in enumerate(ids):
                scores[cid] += 1.0 / (k + rank + 1)
        # Sort: high score first, deterministic tie-break by id
        return sorted(scores.items(), key=lambda x: (-x[1], x[0]))

    # --------------------------------------------------------------------- #
    # 5) Fetch the chunk text                                               #
    # --------------------------------------------------------------------- #
    def _hydrate_chunks(self, ids: List[str]) -> List[Dict]:
        if not ids:
            return []
        cypher = (
            "MATCH (c:Chunk) WHERE c.id IN $ids "
            "RETURN c.id AS id, c.text AS text"
        )
        with self.driver.session() as s:
            rows = s.run(cypher, ids=ids).data()
        lookup = {r["id"]: r["text"] for r in rows}
        return [{"id": cid, "text": lookup[cid]} for cid in ids if cid in lookup]

    # --------------------------------------------------------------------- #
    # Fallback entity tagger                                                #
    # --------------------------------------------------------------------- #
    @staticmethod
    def _naive_capitalised_np(text: str) -> List[str]:
        """Very small regex-based heuristic: sequences of ≥2 capitalised words."""
        import re

        patt = re.compile(r"\b([A-Z][\w\-]*(?:\s+[A-Z][\w\-]*)+)\b")
        return patt.findall(text)
    
    def get_supporting_triples(self, chunk_ids: List[str]) -> List[Dict]:
        if not chunk_ids:
            return []

        cypher = """
        MATCH (c:Chunk)
        WHERE c.id IN $chunk_ids
        OPTIONAL MATCH (e1)-[r]->(e2)
        WHERE (e1)-[:MENTIONED_IN]->(c) AND (e2)-[:MENTIONED_IN]->(c)
        RETURN DISTINCT e1.name AS subject, type(r) AS relation, e2.name AS object
        LIMIT 30
        """
        with self.driver.session() as session:
            rows = session.run(cypher, chunk_ids=chunk_ids).data()
        return rows
    
        # --------------------------------------------------------------------- #
    # Public API
    # --------------------------------------------------------------------- #
    def invoke(self, query: str, entities: Optional[List[str]] = None) -> Dict[str, List[Dict]]:
        dense_ids, _ = self._dense_seeds(query)

        expansion_ids = (
            self._expand_parents(dense_ids, self.expand_parents)
            | self._expand_neighbors(dense_ids, self.expand_neighbors)
        )

        if entities is None:
            entities = self.entity_tagger(query)

        kg_ids = self.kg_probe_from_entities(entities)

        fused = self._rrf_fuse(
            [
                ("dense", dense_ids),
                ("expand", list(expansion_ids)),
                ("kg", kg_ids),
            ]
        )

        final_ids = [cid for cid, _ in fused[: self.max_chunks]]
        chunks = self._hydrate_chunks(final_ids)
        triples = self.get_supporting_triples(final_ids)

        return {
            "chunks": chunks,
            "triples": triples,
        }


In [10]:
class SmartRAGPipeline:
    def __init__(self, hybrid_retriever, triple_retriever, llm):
        self.hybrid = hybrid_retriever
        self.triples = triple_retriever
        self.llm = llm

    def run(self, query: str) -> str:
        entities = extract_entities(query)
        intent = classify_intent(query)

        if intent == "fact":
            # Mode A: KG-only
            triples = self.triples.get_triples(entities)
            triple_text = self._format_triples(triples)
            prompt = f"""Answer using the facts below. If the answer isn't found, say "I don't know."\n\n{triple_text}\n\nQ: {query}\nA:"""
            return self.llm(prompt)

        else:
            # Mode B: Chunks + triples
            chunks = self.hybrid.invoke(query, entities=entities)
            chunk_ids = [c["id"] for c in chunks]
            chunk_text = "\n\n---\n\n".join([c["text"] for c in chunks])
            triples = self.hybrid.get_supporting_triples(chunk_ids)
            triple_text = self._format_triples(triples)

            prompt = f"""Use the facts and context to answer. If not answerable, say "I don't know."\n\nFacts:\n{triple_text}\n\nContext:\n{chunk_text}\n\nQ: {query}\nA:"""
            return self.llm(prompt)

    def _format_triples(self, triples: List[Dict]) -> str:
        return "\n".join(f"- ({t['subject']}, {t['relation']}, {t['object']})" for t in triples)


In [16]:
retriever = HybridRetriever(
    driver=driver,
    embedder=embedder,
    index_name="chunk-index",
)

In [26]:
query = "List all iPhones released by Apple in 2022"

context = retriever.invoke(query)

In [27]:
context

{'chunks': [{'id': 'chunk_0035',
   'text': 'COVID-19\nThe COVID-19 pandemic has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented\nmeasures, such as restrictions on travel and business operations, temporary closures of businesses, and quarantine and shelter-in-place orders. The COVID-19\npandemic has at times significantly curtailed global economic activity and caused significant volatility and disruption in global financial markets. The COVID-19\npandemic and the measures taken by many countries in response have affected and could in the future materially impact the Company’s business, results of\noperations and financial condition.\nCertain of the Company’s outsourcing partners, component suppliers and logistical service providers have experienced disruptions during the COVID-19\npandemic, resulting in supply shortages. Similar disruptions could occur in the future.\nProducts and Services Performance\nThe fo

In [28]:
context['chunks']

[{'id': 'chunk_0035',
  'text': 'COVID-19\nThe COVID-19 pandemic has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented\nmeasures, such as restrictions on travel and business operations, temporary closures of businesses, and quarantine and shelter-in-place orders. The COVID-19\npandemic has at times significantly curtailed global economic activity and caused significant volatility and disruption in global financial markets. The COVID-19\npandemic and the measures taken by many countries in response have affected and could in the future materially impact the Company’s business, results of\noperations and financial condition.\nCertain of the Company’s outsourcing partners, component suppliers and logistical service providers have experienced disruptions during the COVID-19\npandemic, resulting in supply shortages. Similar disruptions could occur in the future.\nProducts and Services Performance\nThe following tabl

In [29]:
context['triples']

[{'subject': 'FinancialItem',
  'relation': 'RELATEDTOPRODUCT',
  'object': 'Product'},
 {'subject': 'FinancialItem',
  'relation': 'COVERSPERIOD',
  'object': 'DatePeriod'},
 {'subject': 'Product', 'relation': 'HASLINEITEM', 'object': 'FinancialItem'},
 {'subject': 'Company',
  'relation': 'HASSTATEMENT',
  'object': 'FinancialReport'},
 {'subject': 'FinancialReport',
  'relation': 'HASLINEITEM',
  'object': 'FinancialItem'},
 {'subject': 'FinancialReport',
  'relation': 'COVERSPERIOD',
  'object': 'DatePeriod'},
 {'subject': 'Apple Inc.', 'relation': 'HASLINEITEM', 'object': 'Q3 2022'},
 {'subject': 'Apple Inc.',
  'relation': 'HASLINEITEM',
  'object': 'Dividends Paid'},
 {'subject': 'Apple Inc.',
  'relation': 'HASLINEITEM',
  'object': 'Stock Repurchase'},
 {'subject': 'Apple Inc.',
  'relation': 'HASSTATEMENT',
  'object': 'Q3 2022 Form 10-Q'},
 {'subject': 'Apple Inc.',
  'relation': 'REPORTSON',
  'object': 'Q3 2022 Form 10-Q'},
 {'subject': 'Apple Inc.',
  'relation': 'RELATED

#### RAG PIPELINE

In [ ]:
def format_triples(triples: List[Dict]) -> str:
    return "\n".join(f"- ({t['subject']}, {t['relation']}, {t['object']})" for t in triples)

In [ ]:
query = "How was the apple sales in 2022?"
entities = extract_entities(query)
context = retriever.invoke(query, entities)

chunk_text = "\n\n---\n\n".join([c["text"] for c in context['chunks']])
chunk_ids = [c["id"] for c in context['chunks']]
triples = context['triples']
triple_text = format_triples(triples)

In [ ]:
prompt = f"""Use the facts and context to answer. If not answerable, say "I don't know."\n\nFacts:\n{triple_text}\n\nContext:\n{chunk_text}\n\nQ: {query}\nA:"""
print(prompt)

Use the facts and context to answer. If not answerable, say "I don't know."

Facts:
- (Apple, HASSTATEMENT, Net Sales)
- (Net Sales, HASLINEITEM, Services)
- (Net Sales, HASLINEITEM, Wearables, Home and Accessories)
- (Net Sales, HASLINEITEM, iPad)
- (Net Sales, HASLINEITEM, Mac)
- (Net Sales, HASLINEITEM, iPhone)
- (Total net sales, HASVALUE, 82,959)
- (Apple Inc., HASSTATEMENT, Q3 2022 Form 10-Q)
- (Q3 2022 Form 10-Q, HASLINEITEM, Revenue Recognized)
- (Q3 2022 Form 10-Q, HASLINEITEM, Deferred Revenue)
- (FinancialItem, RELATEDTOSERVICE, Service)
- (FinancialItem, RELATEDTOPRODUCT, Product)
- (Product, HASLINEITEM, FinancialItem)
- (FinancialReport, HASSTATEMENT, FinancialStatement)
- (FinancialReport, HASLINEITEM, FinancialItem)
- (iPhone, RELATEDTOSERVICE, Services)
- (iPhone, HASVALUE, 28,669)
- (Mac, HASVALUE, 26,012)
- (Wearables, Home and Accessories, HASVALUE, 22,118)
- (Services, HASVALUE, 19,604)
- (Apple Inc., HASLINEITEM, Services)
- (Apple Inc., RELATEDTOSERVICE, Services

In [ ]:
response = llm(prompt)
print(response)

content="In 2022, Apple's total net sales for the third quarter amounted to $82,959 million, showing a 2% increase compared to $81,434 million in the same quarter of the previous year. The iPhone generated the highest sales at $28,669 million, followed by the Mac at $26,012 million, services at $19,604 million, and wearables, home and accessories at $22,118 million. However, both iPad and wearables, home and accessories experienced decreased net sales compared to the previous year. Overall, services sales increased by 12%, indicating strong growth in that segment." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 3306, 'total_tokens': 3435, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', '

#### Problems

-  
- 

-

## Retriever = Approach - 2

In [ ]:
from typing import List, Dict, Optional, Any
import numpy as np
from neo4j import GraphDatabase
from langchain.embeddings.base import Embeddings
from openai import OpenAI
import json
from enum import Enum, auto

class QueryType(Enum):
    ENTITY_FOCUSED = auto()  # "Who is Tim Cook?"
    RELATIONSHIP = auto()    # "What products did Apple release in 2023?"
    COMPARISON = auto()      # "Compare iPhone 13 and iPhone 14"
    CONTEXTUAL = auto()      # "Explain how Apple's supply chain works"
    TEMPORAL = auto()        # "What happened to Apple stock after the iPhone 15 launch?"

class EnhancedGraphRetriever:
    """
    An adaptive retriever that leverages both graph structure and semantic similarity,
    with dynamic weighting based on query understanding.
    """
    
    def __init__(
        self,
        driver: GraphDatabase.driver,
        embedder: Embeddings,
        llm_client: OpenAI,
        *,
        index_name: str,
        max_chunks: int = 12,
        max_triples: int = 20,
    ):
        self.driver = driver
        self.embedder = embedder
        self.llm_client = llm_client
        self.index_name = index_name
        self.max_chunks = max_chunks
        self.max_triples = max_triples
    
    def _classify_query(self, query: str) -> Dict[str, Any]:
        """Analyze query to determine type, entities, and relevant properties"""
        tools = [{
            "type": "function",
            "function": {
                "name": "analyze_query",
                "description": "Analyzes a search query for RAG retrieval",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query_type": {
                            "type": "string",
                            "enum": [qt.name for qt in QueryType],
                            "description": "The type of query"
                        },
                        "entities": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Named entities in the query"
                        },
                        "relationships": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Types of relationships to look for"
                        },
                        "time_focus": {
                            "type": "string",
                            "description": "Any temporal focus in the query (e.g., '2023', 'recent', 'after launch')"
                        },
                        "attributes": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Key attributes or properties mentioned"
                        }
                    },
                    "required": ["query_type", "entities"]
                }
            }
        }]

        response = self.llm_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Analyze the search query to optimize knowledge graph retrieval."},
                {"role": "user", "content": query}
            ],
            tools=tools,
            tool_choice={"type": "function", "function": {"name": "analyze_query"}}
        )

        args = response.choices[0].message.tool_calls[0].function.arguments
        return json.loads(args)
    
    def _retrieve_entity_context(self, entities: List[str], limit: int = 10) -> List[Dict]:
        """Get chunks that provide context about specified entities"""
        cypher = """
        MATCH (e)
        WHERE toLower(e.name) IN $entities
        MATCH (e)-[:MENTIONED_IN]->(c:Chunk)
        WITH c, count(DISTINCT e) AS entity_matches
        ORDER BY entity_matches DESC
        LIMIT $limit
        RETURN c.id AS id, c.text AS text, entity_matches
        """
        
        with self.driver.session() as session:
            rows = session.run(
                cypher,
                entities=[e.lower() for e in entities],
                limit=limit
            ).data()
        
        return rows
    
    def _retrieve_relationship_context(
        self, 
        entities: List[str], 
        relationships: List[str] = None,
        limit: int = 10
    ) -> List[Dict]:
        """Get chunks describing relationships between entities"""
        
        # Base query when no specific relationships are provided
        if not relationships or len(relationships) == 0:
            cypher = """
            MATCH (e1)-[r]->(e2)
            WHERE toLower(e1.name) IN $entities OR toLower(e2.name) IN $entities
            MATCH (e1)-[:MENTIONED_IN]->(c:Chunk)<-[:MENTIONED_IN]-(e2)
            RETURN DISTINCT c.id AS id, c.text AS text, 
                   count(DISTINCT r) AS rel_count
            ORDER BY rel_count DESC
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "limit": limit
            }
        else:
            # Query with specific relationship types
            cypher = """
            MATCH (e1)-[r]->(e2)
            WHERE (toLower(e1.name) IN $entities OR toLower(e2.name) IN $entities)
                  AND type(r) IN $relationships
            MATCH (e1)-[:MENTIONED_IN]->(c:Chunk)<-[:MENTIONED_IN]-(e2)
            RETURN DISTINCT c.id AS id, c.text AS text, 
                   count(DISTINCT r) AS rel_count
            ORDER BY rel_count DESC
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "relationships": [r.upper().replace(' ', '_') for r in relationships],
                "limit": limit
            }
            
        with self.driver.session() as session:
            rows = session.run(cypher, **params).data()
            
        return rows
    
    def _retrieve_temporal_context(
        self, 
        entities: List[str],
        time_focus: str,
        limit: int = 10
    ) -> List[Dict]:
        """Get chunks with temporal focus"""
        # First try to find chunks with explicit time references
        cypher = """
        MATCH (e)
        WHERE toLower(e.name) IN $entities
        MATCH (e)-[:MENTIONED_IN]->(c:Chunk)
        WHERE c.text CONTAINS $time_focus
        RETURN c.id AS id, c.text AS text, 1.0 AS score
        LIMIT $limit
        """
        
        with self.driver.session() as session:
            rows = session.run(
                cypher,
                entities=[e.lower() for e in entities],
                time_focus=time_focus,
                limit=limit
            ).data()
            
        return rows
    
    def _retrieve_dense_vectors(self, query: str, limit: int = 10) -> List[Dict]:
        """Semantic search using embeddings"""
        embedding = self.embedder.embed_query(query)
        cypher = """
        CALL db.index.vector.queryNodes($index_name, $k, $embed) 
        YIELD node, score 
        RETURN node.id AS id, node.text AS text, score
        ORDER BY score DESC
        """
        
        with self.driver.session() as session:
            rows = session.run(
                cypher,
                index_name=self.index_name,
                k=limit,
                embed=embedding
            ).data()
            
        return rows
    
    def _retrieve_comparison_context(
        self,
        entities: List[str],
        attributes: List[str] = None,
        limit: int = 10
    ) -> List[Dict]:
        """Get chunks comparing entities or discussing specific attributes"""
        # Base query when no attributes specified
        if not attributes or len(attributes) == 0:
            cypher = """
            MATCH (e1)-[:MENTIONED_IN]->(c:Chunk)<-[:MENTIONED_IN]-(e2)
            WHERE toLower(e1.name) IN $entities AND toLower(e2.name) IN $entities 
                  AND e1 <> e2
            RETURN c.id AS id, c.text AS text, 
                   count(DISTINCT e1) + count(DISTINCT e2) AS entity_count
            ORDER BY entity_count DESC
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "limit": limit
            }
        else:
            # Try to find specific attribute mentions
            attribute_clauses = []
            for attr in attributes:
                attribute_clauses.append(f"c.text CONTAINS '{attr}'")
            
            attr_filter = " OR ".join(attribute_clauses)
            
            cypher = f"""
            MATCH (e1)-[:MENTIONED_IN]->(c:Chunk)<-[:MENTIONED_IN]-(e2)
            WHERE toLower(e1.name) IN $entities AND toLower(e2.name) IN $entities 
                  AND e1 <> e2
                  AND ({attr_filter})
            RETURN c.id AS id, c.text AS text, 
                   count(DISTINCT e1) + count(DISTINCT e2) AS entity_count
            ORDER BY entity_count DESC
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "limit": limit
            }
            
        with self.driver.session() as session:
            rows = session.run(cypher, **params).data()
            
        return rows
    
    def get_relevant_triples(self, entities: List[str], relationships: List[str] = None) -> List[Dict]:
        """Retrieve knowledge graph triples directly"""
        if not relationships or len(relationships) == 0:
            cypher = """
            MATCH (s)-[r]->(o)
            WHERE toLower(s.name) IN $entities OR toLower(o.name) IN $entities
            RETURN DISTINCT s.name AS subject, type(r) AS relation, o.name AS object
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "limit": self.max_triples
            }
        else:
            cypher = """
            MATCH (s)-[r]->(o)
            WHERE (toLower(s.name) IN $entities OR toLower(o.name) IN $entities)
                  AND type(r) IN $relationships
            RETURN DISTINCT s.name AS subject, type(r) AS relation, o.name AS object
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "relationships": [r.upper().replace(' ', '_') for r in relationships],
                "limit": self.max_triples
            }
            
        with self.driver.session() as session:
            rows = session.run(cypher, **params).data()
            
        return rows
    
    def _merge_and_deduplicate(self, chunk_sets: List[List[Dict]]) -> List[Dict]:
        """Merge multiple chunk sets with scoring and deduplication"""
        id_to_chunk = {}
        id_to_score = {}
        
        # Process each chunk set with different weights based on position
        for i, chunk_set in enumerate(chunk_sets):
            # Weight earlier sets higher (they're more precisely matched to query type)
            weight = 1.0 / (i + 1)
            
            for chunk in chunk_set:
                chunk_id = chunk["id"]
                
                # Store the chunk if we haven't seen it
                if chunk_id not in id_to_chunk:
                    id_to_chunk[chunk_id] = chunk
                    id_to_score[chunk_id] = 0
                
                # Add to the score (different chunk sets may score differently)
                if "score" in chunk:
                    id_to_score[chunk_id] += chunk["score"] * weight
                elif "entity_matches" in chunk:
                    id_to_score[chunk_id] += chunk["entity_matches"] * weight
                elif "rel_count" in chunk:
                    id_to_score[chunk_id] += chunk["rel_count"] * weight
                elif "entity_count" in chunk:
                    id_to_score[chunk_id] += chunk["entity_count"] * weight
                else:
                    id_to_score[chunk_id] += weight
        
        # Sort by score and return top chunks
        sorted_ids = sorted(id_to_score.keys(), key=lambda x: -id_to_score[x])
        return [id_to_chunk[chunk_id] for chunk_id in sorted_ids[:self.max_chunks]]
    
    def invoke(self, query: str) -> Dict[str, Any]:
        """Main retrieval method adapting to query type"""
        # Analyze the query
        query_analysis = self._classify_query(query)
        query_type = QueryType[query_analysis["query_type"]]
        entities = query_analysis["entities"]
        
        # Initialize chunk lists based on query type
        chunks_from_specialized = []
        chunks_from_dense = self._retrieve_dense_vectors(query)
        
        # Apply specialized retrievers based on query type
        if query_type == QueryType.ENTITY_FOCUSED:
            chunks_from_specialized = self._retrieve_entity_context(entities)
            
        elif query_type == QueryType.RELATIONSHIP:
            relationships = query_analysis.get("relationships", [])
            chunks_from_specialized = self._retrieve_relationship_context(
                entities, relationships
            )
            
        elif query_type == QueryType.COMPARISON:
            attributes = query_analysis.get("attributes", [])
            chunks_from_specialized = self._retrieve_comparison_context(
                entities, attributes
            )
            
        elif query_type == QueryType.TEMPORAL:
            time_focus = query_analysis.get("time_focus", "")
            if time_focus:
                chunks_from_specialized = self._retrieve_temporal_context(
                    entities, time_focus
                )
            
        # For contextual queries, we rely more on dense retrieval
        # But for all types, we merge specialized and dense results
        merged_chunks = self._merge_and_deduplicate([
            chunks_from_specialized,
            chunks_from_dense
        ])
        
        # Get relevant triples
        relationships = query_analysis.get("relationships", [])
        triples = self.get_relevant_triples(entities, relationships)
        
        # Return both chunks and triples
        return {
            "chunks": merged_chunks,
            "triples": triples,
            "query_analysis": query_analysis
        }

In [ ]:
# Setup
llm_client = OpenAI(api_key=api_key)

# Create retriever
retriever = EnhancedGraphRetriever(
    driver=driver,
    embedder=embedder,
    llm_client=llm_client,
    index_name="chunk-index"
)

# Example queries
result1 = retriever.invoke("Who is Tim Cook?")
result2 = retriever.invoke("What products did Apple release in 2023?")
result3 = retriever.invoke("Compare iPhone 13 and iPhone 14 battery life")
result4 = retriever.invoke("How does Apple's supply chain work?")

In [ ]:
result1.keys()

dict_keys(['chunks', 'triples', 'query_analysis'])

In [ ]:
print(result1['chunks'])
print(result1['triples'])
print(result1['query_analysis'])

[{'id': 'chunk_0057', 'text': 'Exhibit 31.1\nCERTIFICATION\nI, Timothy D. Cook, certify that:\n1.\nI have reviewed this quarterly report on Form 10-Q of Apple Inc.;\n2.\nBased on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to make the\nstatements made, in light of the circumstances under which such statements were made, not misleading with respect to the period covered by this report;\n3.\nBased on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects the financial\ncondition, results of operations and cash flows of the Registrant as of, and for, the periods presented in this report;\n4.\nThe Registrant’s other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as defined in Exchange\nAct Rules 13a-15(e) and 15d-15(e)) and internal control over financial reportin

In [ ]:
print(result2['chunks'])
print(result2['triples'])
print(result2['query_analysis'])

[{'id': 'chunk_0015', 'text': 'Note 2 – Revenue\nNet sales disaggregated by significant products and services for the three- and nine-month periods ended June 25, 2022 and June\xa026, 2021 were as follows (in\nmillions):\nThree Months Ended\nNine Months Ended\nJune 25,\n2022\nJune 26,\n2021\nJune 25,\n2022\nJune 26,\n2021\niPhone\n$\n40,665\xa0\n$\n39,570\xa0\n$\n162,863\xa0\n$\n153,105\xa0\nMac  \n7,382\xa0\n8,235\xa0\n28,669\xa0\n26,012\xa0\niPad\n7,224\xa0\n7,368\xa0\n22,118\xa0\n23,610\xa0\nWearables, Home and Accessories \n8,084\xa0\n8,775\xa0\n31,591\xa0\n29,582\xa0\nServices \n19,604\xa0\n17,486\xa0\n58,941\xa0\n50,148\xa0\nTotal net sales \n$\n82,959\xa0\n$\n81,434\xa0\n$\n304,182\xa0\n$\n282,457\xa0\n(1)\nProducts net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective\nproduct.\n(2)\nWearables, Home and Accessories net sales include sales of AirPods , Apple TV , Apple Watch , Beats  pr

## Retriever = Approach - 3

In [ ]:
from typing import List, Dict, Any
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from openai import OpenAI

class GraphRAGChain:
    """
    End-to-end chain that:
    1. Analyzes the query to determine intent
    2. Retrieves relevant chunks and triples from the knowledge graph
    3. Formats the context with both textual chunks and structured triples
    4. Generates a response using the retrieved context
    """
    
    def __init__(
        self,
        retriever,
        llm_client: OpenAI,
        model_name: str = "gpt-4o",
        use_structured_output: bool = True
    ):
        self.retriever = retriever
        self.llm_client = llm_client
        self.model_name = model_name
        self.use_structured_output = use_structured_output
        
    def _format_triples(self, triples: List[Dict]) -> str:
        """Format knowledge graph triples in a readable way"""
        if not triples:
            return ""
            
        formatted = "KNOWLEDGE GRAPH FACTS:\n"
        for i, t in enumerate(triples):
            formatted += f"- {t['subject']} → {t['relation'].replace('_', ' ').title()} → {t['object']}\n"
        
        return formatted
    
    def _format_context(self, result: Dict[str, Any]) -> str:
        """Format both chunks and triples into a single context string"""
        chunks = result.get("chunks", [])
        triples = result.get("triples", [])
        query_analysis = result.get("query_analysis", {})
        
        formatted_chunks = ""
        for i, chunk in enumerate(chunks):
            chunk_id = chunk.get("id", f"chunk_{i}")
            # Extract metadata if available
            metadata = {}
            if "_" in chunk_id:
                parts = chunk_id.split("_")
                if len(parts) >= 3:
                    metadata["source"] = parts[0]
                    metadata["page"] = parts[1] if len(parts) > 1 else "unknown"
            
            # Format the chunk with metadata
            formatted_chunks += f"\n--- CHUNK {i+1} ---\n"
            if metadata:
                formatted_chunks += f"Source: {metadata.get('source', 'unknown')}, Page: {metadata.get('page', 'unknown')}\n"
            formatted_chunks += f"{chunk['text']}\n"
        
        # Format triples
        formatted_triples = self._format_triples(triples)
        
        # Format query analysis if available
        query_info = ""
        if query_analysis:
            query_type = query_analysis.get("query_type", "")
            entities = query_analysis.get("entities", [])
            
            if query_type and entities:
                query_info = f"\nQUERY ANALYSIS:\n- Type: {query_type}\n- Entities: {', '.join(entities)}\n"
        
        # Combine all parts
        return f"{query_info}\n{formatted_triples}\n\nRELEVANT TEXT CHUNKS:{formatted_chunks}"
    
    def _generate_response(self, question: str, context: str) -> str:
        """Generate a response using the LLM"""
        if self.use_structured_output:
            return self._generate_structured_response(question, context)
        else:
            return self._generate_simple_response(question, context)
    
    def _generate_simple_response(self, question: str, context: str) -> str:
        """Generate a simple text response"""
        prompt_template = """
        You are an intelligent assistant with access to a knowledge graph and document chunks.
        Answer the question based on the context provided.
        
        If the information isn't in the context, say "I don't have enough information to answer this question."
        
        When using information from the context, cite your sources with (Source: <source>, Page: <page>) at the end of sentences.
        
        Question: {question}
        
        Context:
        {context}
        
        Answer:
        """
        
        prompt = ChatPromptTemplate.from_template(prompt_template)
        
        response = self.llm_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(question=question, context=context)}
            ]
        )
        
        return response.choices[0].message.content
    
    def _generate_structured_response(self, question: str, context: str) -> str:
        """Generate a structured response with citations and explanations"""
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "answer_with_citations",
                    "description": "Answer questions with citations from the provided context",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "answer": {
                                "type": "string",
                                "description": "The comprehensive answer to the question"
                            },
                            "citations": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "text": {
                                            "type": "string",
                                            "description": "The text being cited"
                                        },
                                        "source": {
                                            "type": "string",
                                            "description": "Source document"
                                        },
                                        "page": {
                                            "type": "string",
                                            "description": "Page number or section"
                                        }
                                    }
                                },
                                "description": "Citations supporting the answer"
                            },
                            "knowledge_graph_facts_used": {
                                "type": "array",
                                "items": {
                                    "type": "string"
                                },
                                "description": "Facts from the knowledge graph used in the answer"
                            },
                            "confidence": {
                                "type": "string",
                                "enum": ["high", "medium", "low"],
                                "description": "Confidence level in the answer based on the available context"
                            }
                        },
                        "required": ["answer", "citations", "confidence"]
                    }
                }
            }
        ]
        
        prompt_content = f"""
        Answer the question based on the provided context.
        
        The context includes both text chunks and knowledge graph facts. Use both to provide a comprehensive answer.
        
        Question: {question}
        
        Context:
        {context}
        
        Provide a detailed answer with appropriate citations to the sources. 
        If the information isn't in the context, indicate that you don't have enough information.
        """
        
        response = self.llm_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant with access to both document text and a knowledge graph."},
                {"role": "user", "content": prompt_content}
            ],
            tools=tools,
            tool_choice={"type": "function", "function": {"name": "answer_with_citations"}}
        )
        
        try:
            # Parse the structured response
            tool_call = response.choices[0].message.tool_calls[0]
            result = json.loads(tool_call.function.arguments)
            
            # Format the final answer with citations
            formatted_answer = result["answer"]
            
            # Add citations section
            if result["citations"]:
                formatted_answer += "\n\nSources:"
                for i, citation in enumerate(result["citations"]):
                    formatted_answer += f"\n[{i+1}] {citation['source']}"
                    if citation.get('page'):
                        formatted_answer += f", Page: {citation['page']}"
            
            # Add knowledge graph facts used
            if result.get("knowledge_graph_facts_used"):
                formatted_answer += "\n\nKnowledge Graph Facts Used:"
                for fact in result["knowledge_graph_facts_used"]:
                    formatted_answer += f"\n- {fact}"
            
            # Add confidence level
            formatted_answer += f"\n\nConfidence: {result['confidence'].capitalize()}"
            
            return formatted_answer
            
        except (KeyError, IndexError, json.JSONDecodeError) as e:
            # Fallback to using the direct message content
            return response.choices[0].message.content
    
    def invoke(self, query: str) -> str:
        """Execute the full chain to get a response for the query"""
        # Step 1: Retrieve relevant information
        retrieval_result = self.retriever.invoke(query)
        
        # Step 2: Format the context
        formatted_context = self._format_context(retrieval_result)
        
        # Step 3: Generate the response
        response = self._generate_response(query, formatted_context)
        
        return response

In [ ]:
# Usage example
def create_graph_rag_chain(driver, embedder, llm_client):
    from enum import Enum, auto
    
    # Use the EnhancedGraphRetriever from previous code
    retriever = EnhancedGraphRetriever(
        driver=driver,
        embedder=embedder,
        llm_client=llm_client,
        index_name="chunk-index"
    )
    
    # Create the chain
    chain = GraphRAGChain(
        retriever=retriever,
        llm_client=llm_client,
        model_name="gpt-4o-mini",
        use_structured_output=True
    )
    
    return chain

# Create chain
rag_chain = create_graph_rag_chain(driver, embedder, llm_client)

In [ ]:
# Use the chain
query = "What were Apple's revenue figures for Q3 2022?"
response = rag_chain.invoke(query)
print(response)

Apple's total revenue (net sales) for the third quarter (Q3) of 2022, which ended on June 25, 2022, was $82,959 million (or approximately $83 billion). This figure represents an increase of 2% compared to the same quarter in 2021, driven primarily by higher net sales of services and iPhone, though it was partially offset by lower sales of Mac and Wearables, Home and Accessories. The net sales breakdown for the quarter also showed that iPhone sales were $40,665 million, Mac sales were $7,382 million, iPad sales $7,224 million, Wearables, Home and Accessories sales were $8,084 million, and Services revenue stood at $19,604 million.

Sources:
[1] Apple Inc. | Q3 2022 Form 10-Q, Page: 7
[2] Apple Inc. | Q3 2022 Form 10-Q, Page: 8
[3] Apple Inc. | Q3 2022 Form 10-Q, Page: 8
[4] Apple Inc. | Q3 2022 Form 10-Q, Page: 8
[5] Apple Inc. | Q3 2022 Form 10-Q, Page: 8
[6] Apple Inc. | Q3 2022 Form 10-Q, Page: 8

Confidence: High


## Retriever = Approach - 3

Chunks_With_summaries

In [ ]:
class EnhancedGraphRetriever:
    """
    A minimally modified version of your original retriever that adds source tracking
    """
    
    def __init__(
        self,
        driver,
        embedder,
        llm_client,
        *,
        index_name="chunk-index",
        max_chunks=12,
        max_triples=20,
    ):
        self.driver = driver
        self.embedder = embedder
        self.llm_client = llm_client
        self.index_name = index_name
        self.max_chunks = max_chunks
        self.max_triples = max_triples
    
    def _classify_query(self, query: str):
        """Analyze query to determine type, entities, and relevant properties"""
        tools = [{
            "type": "function",
            "function": {
                "name": "analyze_query",
                "description": "Analyzes a search query for RAG retrieval",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query_type": {
                            "type": "string",
                            "enum": ["ENTITY_FOCUSED", "RELATIONSHIP", "COMPARISON", "CONTEXTUAL", "TEMPORAL"],
                            "description": "The type of query"
                        },
                        "entities": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Named entities in the query"
                        },
                        "relationships": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "Types of relationships to look for"
                        }
                    },
                    "required": ["query_type", "entities"]
                }
            }
        }]

        try:
            response = self.llm_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Analyze the search query to optimize knowledge graph retrieval."},
                    {"role": "user", "content": query}
                ],
                tools=tools,
                tool_choice={"type": "function", "function": {"name": "analyze_query"}}
            )

            args = response.choices[0].message.tool_calls[0].function.arguments
            return json.loads(args)
        except Exception as e:
            print(f"Error classifying query: {e}")
            # Fallback
            return {
                "query_type": "CONTEXTUAL",
                "entities": [],
                "relationships": []
            }
    
    def _check_for_summary_request(self, query):
        """Check if query is requesting a summary"""
        summary_keywords = ["summary", "overview", "summarize", "summarise", "entire dataset"]
        return any(keyword in query.lower() for keyword in summary_keywords)
    
    def _retrieve_top_summaries(self, limit=3):
        """Retrieve top-level summary nodes"""
        try:
            cypher = """
            MATCH (s)
            WHERE s.type = 'summary'
            RETURN s.id AS id, s.text AS text, s.type AS type
            LIMIT $limit
            """
            
            with self.driver.session() as session:
                rows = session.run(cypher, limit=limit).data()
            
            return rows
        except Exception as e:
            print(f"Error retrieving summaries: {e}")
            return []
    
    def _retrieve_dense_vectors(self, query, limit=10):
        """Vector search with correct index name"""
        try:
            embedding = self.embedder.embed_query(query)
            cypher = """
            CALL db.index.vector.queryNodes($index_name, $k, $embed) 
            YIELD node, score 
            RETURN node.id AS id, node.text AS text, node.type AS type, score
            ORDER BY score DESC
            """
            
            with self.driver.session() as session:
                rows = session.run(
                    cypher,
                    index_name=self.index_name,
                    k=limit,
                    embed=embedding
                ).data()
                
            return rows
        except Exception as e:
            print(f"Vector search error: {e}")
            return self._text_search_fallback(query, limit)
    
    def _text_search_fallback(self, query, limit=10):
        """Text-based search fallback"""
        terms = [w.lower() for w in query.split() if len(w) > 3]
        
        if not terms:
            cypher = """
            MATCH (c)
            WHERE c.type = 'chunk'
            RETURN c.id AS id, c.text AS text, c.type AS type, 0.5 AS score
            LIMIT $limit
            """
            params = {"limit": limit}
        else:
            conditions = []
            params = {"limit": limit}
            
            for i, term in enumerate(terms):
                conditions.append(f"toLower(c.text) CONTAINS ${i}")
                params[str(i)] = term
            
            condition_str = " OR ".join(conditions)
            
            cypher = f"""
            MATCH (c)
            WHERE ({condition_str})
            RETURN c.id AS id, c.text AS text, c.type AS type, 0.5 AS score
            LIMIT $limit
            """
        
        try:
            with self.driver.session() as session:
                rows = session.run(cypher, **params).data()
            return rows
        except Exception as e:
            print(f"Text search error: {e}")
            return []
    
    def get_relevant_triples(self, entities, relationships=None):
        """Get knowledge graph triples"""
        if not entities:
            cypher = """
            MATCH (s)-[r]->(o)
            RETURN s.name AS subject, type(r) AS relation, o.name AS object
            LIMIT $limit
            """
            params = {"limit": self.max_triples}
        elif not relationships or len(relationships) == 0:
            cypher = """
            MATCH (s)-[r]->(o)
            WHERE toLower(s.name) IN $entities OR toLower(o.name) IN $entities
            RETURN s.name AS subject, type(r) AS relation, o.name AS object
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "limit": self.max_triples
            }
        else:
            cypher = """
            MATCH (s)-[r]->(o)
            WHERE (toLower(s.name) IN $entities OR toLower(o.name) IN $entities)
                AND type(r) IN $relationships
            RETURN s.name AS subject, type(r) AS relation, o.name AS object
            LIMIT $limit
            """
            params = {
                "entities": [e.lower() for e in entities],
                "relationships": [r.upper().replace(' ', '_') for r in relationships],
                "limit": self.max_triples
            }
        
        try:
            with self.driver.session() as session:
                rows = session.run(cypher, **params).data()
            return rows
        
        except Exception as e:
            print(f"Triple retrieval error: {e}")
            return []
    
    def invoke(self, query):
        """Main retrieval method with better source tracking"""
        try:
            # Analyze the query
            query_analysis = self._classify_query(query)
            entities = query_analysis.get("entities", [])
            
            # Check if this is a summary query
            is_summary_query = self._check_for_summary_request(query)
            
            # Get relevant information based on query type
            if is_summary_query:
                # For summary queries, get top summaries
                summaries = self._retrieve_top_summaries()
                chunks = self._retrieve_dense_vectors(query, limit=5)
            else:
                # For regular queries, focus on relevant chunks
                summaries = []
                chunks = self._retrieve_dense_vectors(query)
            
            # Get knowledge graph triples - FIX HERE
            triples = self.get_relevant_triples(entities)
            
            # Return results with clear identification of source types
            return {
                "summaries": summaries,
                "chunks": chunks,
                "triples": triples,
                "query_analysis": query_analysis
            }
        except Exception as e:
            print(f"Retrieval error: {e}")
            return {
                "summaries": [],
                "chunks": self._text_search_fallback(query),
                "triples": [],
                "query_analysis": {"query_type": "CONTEXTUAL", "entities": []}
            }



In [ ]:
retriever = EnhancedGraphRetriever(
    driver=driver,
    embedder=embedder,
    llm_client=llm_client,
    index_name="chunk-index"
)

In [ ]:
query = "what is the overall summary of the dataset?"
context = retriever.invoke(query)

In [ ]:
print(context.keys())

dict_keys(['summaries', 'chunks', 'triples', 'query_analysis'])


In [ ]:
print(context['summaries'])
print(context['chunks'])
print(context['triples'])
print(context['query_analysis'])

[{'id': 'summary_0000', 'text': "**Summary of Apple Inc. Form 10-Q for the Quarter Ended June 25, 2022**\n\n**General Information:**\n- This document is a quarterly report (Form 10-Q) filed with the U.S. Securities and Exchange Commission (SEC) by Apple Inc. for the period ending June 25, 2022.\n- Apple Inc. is incorporated in California and has its principal executive offices located at One Apple Park Way, Cupertino, California.\n\n**Securities Information:**\n- Apple’s common stock is traded on the Nasdaq Stock Market under the symbol AAPL.\n- The report lists various classes of notes issued by Apple, including those due in 2022, 2024, 2025, 2026, 2027, 2029, 2031, and 2042.\n\n**Filing Status:**\n- Apple confirms that it has filed all required reports under the Securities Exchange Act of 1934 and has been subject to these filing requirements for the past 90 days.\n- The company is classified as a large accelerated filer.\n\n**Financial Overview:**\n- As of July 15, 2022, Apple had 1

In [ ]:
class GraphRAGChainWithSourceTracking:
    """
    Chain that provides detailed source attribution
    """
    
    def __init__(
        self,
        retriever,
        llm_client,
        model_name="gpt-4o-mini"
    ):
        self.retriever = retriever
        self.llm_client = llm_client
        self.model_name = model_name
    
    def _format_context(self, result):
        """Format with clear source identification"""
        summaries = result.get("summaries", [])
        chunks = result.get("chunks", [])
        triples = result.get("triples", [])
        
        formatted_text = ""
        
        # Format summaries
        if summaries:
            formatted_text += "SUMMARIES:\n"
            for i, summary in enumerate(summaries):
                summary_id = summary.get("id", f"summary_{i}")
                formatted_text += f"[SUMMARY_{i+1}] ID: {summary_id}\n{summary.get('text', '')}\n\n"
        
        # Format chunks
        if chunks:
            formatted_text += "CHUNKS:\n"
            for i, chunk in enumerate(chunks):
                chunk_id = chunk.get("id", f"chunk_{i}")
                chunk_type = chunk.get("type", "chunk")
                
                # Extract source and page if available
                source = "unknown"
                page = "unknown"
                if chunk_id and isinstance(chunk_id, str) and "_" in chunk_id:
                    parts = chunk_id.split("_")
                    if len(parts) >= 1:
                        source = parts[0]
                    if len(parts) >= 2:
                        page = parts[1]
                
                formatted_text += f"[CHUNK_{i+1}] ID: {chunk_id}, Type: {chunk_type}, Source: {source}, Page: {page}\n"
                formatted_text += f"{chunk.get('text', '')}\n\n"
        
        # Format triples
        if triples:
            formatted_text += "KNOWLEDGE GRAPH TRIPLES:\n"
            for i, triple in enumerate(triples):
                subject = triple.get("subject", "")
                relation = triple.get("relation", "").replace("_", " ").title()
                object_val = triple.get("object", "")
                formatted_text += f"[TRIPLE_{i+1}] {subject} → {relation} → {object_val}\n"
        
        return formatted_text
    
    def invoke(self, query):
        """Generate response with clear source attribution"""
        try:
            # Retrieve relevant information
            retrieval_result = self.retriever.invoke(query)
            
            # Format the context
            formatted_context = self._format_context(retrieval_result)
            
            # Generate response
            prompt = f"""
            You are a financial assistant with access to document chunks, summaries, and knowledge graph facts.
            
            Answer the question based on the context provided.
            If you don't have enough information, say "I don't have enough information to answer this question."
            
            IMPORTANT: In your answer, you MUST include a "Sources:" section at the end that lists ALL sources used:
            1. If you used a SUMMARY, list it as "Summary: [summary ID]"
            2. If you used CHUNKS, list them as "Source: [source name], Page: [page number]"
            3. If you used TRIPLES, list them as "Knowledge Graph: [triple numbers used]"
            
            Question: {query}
            
            Context:
            {formatted_context}
            
            Provide a comprehensive answer with the required source attribution.
            """
            
            response = self.llm_client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant with access to document text and a knowledge graph."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            return response.choices[0].message.content
        except Exception as e:
            return f"I encountered an error while processing your request: {str(e)}"


In [ ]:
retriever = EnhancedGraphRetriever(
    driver=driver,
    embedder=embedder,
    llm_client=llm_client,
    index_name="chunk-index"
)

In [ ]:
chain = GraphRAGChainWithSourceTracking(
    retriever=retriever,
    llm_client=llm_client
)

In [ ]:
query = "what were the apple product sale for each product?"
response = chain.invoke(query)
print(response)

The sales for Apple products in the three-month period ended June 25, 2022, were as follows (in millions):

1. **iPhone**: $40,665
2. **Mac**: $7,382
3. **iPad**: $7,224
4. **Wearables, Home and Accessories**: $8,084
5. **Services**: $19,604

For the nine-month period ended June 25, 2022, the sales figures were:

1. **iPhone**: $162,863
2. **Mac**: $28,669
3. **iPad**: $22,118
4. **Wearables, Home and Accessories**: $31,591
5. **Services**: $58,941

Overall, total net sales for the three months were $82,959 million, and for the nine months, they were $304,182 million.

Sources:
- Source: chunk, Page: 0036
- Source: chunk, Page: 0015
- Source: chunk, Page: 0034
- Source: chunk, Page: 0037


In [ ]:
query = "what is the overall summary of the dataset?"
response = chain.invoke(query)
print(response)

The overall summary of the dataset, which includes Apple Inc.'s financial performance for the quarter ended June 25, 2022, is as follows:

### Summary of Financial Performance
Apple Inc. reported strong financial results for Q3 2022. The highlights are:
- **Net Sales**: Total net sales reached $82.96 billion, marking a slight increase from $81.43 billion in Q3 2021. For the nine months, net sales rose to $304.18 billion from $282.46 billion, reflecting an 8% year-over-year growth.
- **Product and Service Sales**: 
  - iPhone sales increased by 3% to $40.67 billion compared to Q3 2021.
  - Service revenue grew by 12% to $19.60 billion, highlighting a robust demand in this segment. 
  - However, segments like Mac and Wearables experienced a decline in sales.
  
- **Cost of Sales and Expenses**: The total cost of sales increased to $47.07 billion, resulting in a gross margin of $35.89 billion, compared to $35.26 billion in the prior year.
- **Net Income**: The net income for this quarter 